<a href="https://colab.research.google.com/github/aneekbsws/Convolutional-Neural-Networks/blob/main/Resnet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt

#Identity Block#
Identity block is a basic set of three blocks and a residual block.

In [ ]:
from tensorflow.keras.initializers import random_uniform

In [ ]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
  F1, F2, F3 = filters # filters is a tuple containing number of filters of the three layers in main path
  X_short = X

  #1st layer = 1x1 convolution
  X = tf.keras.layers.Conv2D(filters=F1, kernel_size=1, strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
  X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
  X = tf.keras.layers.Activation(activation='relu')(X)

  #Middle layer - fxf convolution with same padding
  X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same', kernel_initializer=initializer(seed=0))(X)
  X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
  X = tf.keras.layers.Activation(activation='relu')(X)

  #last layer - 1x1 convolutio
  X = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
  X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)

  #Add shortcut layer before activation function
  X = tf.keras.layers.Add()([X, X_short])

  #Finally apply activation function
  X = tf.keras.layers.Activation(activation='relu')(X)

  return X

In [ ]:
from tensorflow.keras.initializers import glorot_uniform

In [ ]:
def conv_block(X,f,filters, s=2,training=True, initializer=glorot_uniform):
  F1,F2,F3 = filters
  # print(F1)
  # print(F2)
  # print(F3)
  print(filters)
  X_short = X

  #1st layer = 1x1 convolution, stride = 2
  X = tf.keras.layers.Conv2D(F1, kernel_size=1, strides=(s,s), padding='valid', kernel_initializer=initializer(seed=0))(X) # Will use 
  X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
  X = tf.keras.layers.Activation(activation='relu')(X)

  #Middle layer - fxf convolution with same padding
  X = tf.keras.layers.Conv2D(F2, kernel_size=(f,f), strides=(1,1), padding='same', kernel_initializer=initializer(seed=0))(X)
  X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
  X = tf.keras.layers.Activation(activation='relu')(X)

  #last layer - 1x1 convolution
  X = tf.keras.layers.Conv2D(F3, kernel_size=1, strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
  X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)

  #Shortcut layer - 1x1 convolution, strides = 2
  X_short = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=(s,s), padding='valid', kernel_initializer=initializer(seed=0))(X_short)
  X_short = tf.keras.layers.BatchNormalization(axis=3)(X_short, training=training) 

  #Adding them
  X = tf.keras.layers.Add()([X, X_short])
  X = tf.keras.layers.Activation(activation='relu')(X)

  return X

In [ ]:
from tensorflow.keras.initializers import glorot_uniform

In [ ]:
def resnet_50(input_shape=(64,64,3), classes=6):
  X_input = tf.keras.layers.Input(input_shape)
  X = tf.keras.layers.ZeroPadding2D((3,3))(X_input)

  #Stage 1 -> CONV2D -> BATCHNORM -> RELU -> MaxPooL
  X = tf.keras.layers.Conv2D(64, kernel_size=(7,7), strides=(2,2), kernel_initializer=glorot_uniform(seed=0))(X)
  X = tf.keras.layers.BatchNormalization(axis=3)(X)
  X = tf.keras.layers.Activation(activation='relu')(X)
  X = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(X)

  #Stage-2 -> CONVPOOL -> ID BLOCK*2
  X = conv_block(X, f=3, filters=[64,64,256],s=1)
  X = identity_block(X, 3, [64,64,256])
  X = identity_block(X, 3, [64,64,256])
  #print(X.element_spec)

  #Stage-3 -> CONVPOOL -> ID BLOCK*3
  X = conv_block(X, f=3, filters=[128,128,512],s=2)
  X = identity_block(X, 3, [128,128,512])
  X = identity_block(X, 3, [128,128,512])
  X = identity_block(X, 3, [128,128,512])
  #print(X.element_spec)

  #Stage-4 -> CONVPOOL -> ID-BLOCK*5
  X = conv_block(X, f=3, filters=[256,256,1024],s=2)
  X = identity_block(X, 3, [256,256,1024])
  X = identity_block(X, 3, [256,256,1024])
  X = identity_block(X, 3, [256,256,1024])
  X = identity_block(X, 3, [256,256,1024])
  X = identity_block(X, 3, [256,256,1024])

  #Stage-5 -> CONVPOOL -> ID-BLOCK*2
  X = conv_block(X, f=3, filters=[512,512,2048],s=2)
  X = identity_block(X, 3, [512,512,2048])
  X = identity_block(X, 3, [512,512,2048])

  # Stage-6 -> AVGPOOL -> FLATTEN -> FC
  X = tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=2)(X)
  X = tf.keras.layers.Flatten()(X)
  X = tf.keras.layers.Dense(units = classes, activation='softmax', kernel_initializer=glorot_uniform(seed=0))(X)

  # Create the model
  model = tf.keras.Model(inputs=X_input, outputs=X)

  return model                                    

In [ ]:
model = resnet_50(input_shape=(64,64,3), classes=6)

[64, 64, 256]
[128, 128, 512]
[256, 256, 1024]
[512, 512, 2048]


In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d_22 (ZeroPadding  (None, 70, 70, 3)   0           ['input_25[0][0]']               
 2D)                                                                                              
                                                                                                  
 conv2d_231 (Conv2D)            (None, 32, 32, 64)   9472        ['zero_padding2d_22[0][0]']      
                                                                                                  
 batch_normalization_230 (Batch  (None, 32, 32, 64)  256         ['conv2d_231[0][0]']       

In [ ]:
def load_dataset():
  train_dataset = h5py.File("train_signs.h5","r")
  test_dataset = h5py.File("test_signs.h5")

  train_set_x_orig = np.array(train_dataset["train_set_x"][:])
  train_set_y_orig = np.array(train_dataset["train_set_y"][:])
  test_set_x_orig = np.array(test_dataset["test_set_x"][:])       
  test_set_y_orig = np.array(test_dataset["test_set_y"][:])
  
  classes = np.array(test_dataset["list_classes"][:])
  train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
  test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

  return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
X_train, Y_train, X_test, Y_test, classes = load_dataset()

In [ ]:
def one_hot_encoding(Y,c):
  Y = np.eye(c)[Y.reshape(-1)]
  return Y

In [ ]:
X_train_final = X_train/255.
X_test_final = X_test/255.

Y_train_final = one_hot_encoding(Y_train, c=6)
Y_test_final = one_hot_encoding(Y_test,c=6)
print("X_test : "+ str(X_test_final.shape))
print("X_train : "+str( X_train_final.shape))
print("Y_test : "+ str(Y_test_final.shape))
print("Y_train : "+ str(Y_train_final.shape))

X_test : (120, 64, 64, 3)
X_train : (1080, 64, 64, 3)
Y_test : (120, 6)
Y_train : (1080, 6)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_final, Y_train_final, epochs=10,batch_size=32)

Epoch 1/10
34/34 [==============================] - 188s 5s/step - loss: 1.6500 - accuracy: 0.4917
Epoch 2/10
34/34 [==============================] - 180s 5s/step - loss: 0.6034 - accuracy: 0.7870
Epoch 3/10
34/34 [==============================] - 177s 5s/step - loss: 0.2236 - accuracy: 0.9213
Epoch 4/10
34/34 [==============================] - 176s 5s/step - loss: 0.4235 - accuracy: 0.8824
Epoch 5/10
34/34 [==============================] - 182s 5s/step - loss: 0.2923 - accuracy: 0.9120
Epoch 6/10
34/34 [==============================] - 178s 5s/step - loss: 0.2488 - accuracy: 0.9407
Epoch 7/10
34/34 [==============================] - 180s 5s/step - loss: 0.2692 - accuracy: 0.9157
Epoch 8/10
34/34 [==============================] - 177s 5s/step - loss: 0.1613 - accuracy: 0.9509
Epoch 9/10
34/34 [==============================] - 173s 5s/step - loss: 0.1458 - accuracy: 0.9593
Epoch 10/10
34/34 [==============================] - 179s 5s/step - loss: 0.0872 - accuracy: 0.9769
